In [1]:
import numpy as np
import pandas as pd
import random

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from collections import Counter

Using TensorFlow backend.


In [2]:
csv = pd.read_csv('data.csv', header = None)
data = csv.iloc[:, 0]
target = csv.iloc[:, 1]
target

0     A B A B B B A Z 0 0
1     A C A C C C A Z 0 0
2     A D A D D D A Z 0 0
3     C B C B C B C B Z 0
4     A A A B A A A B Z 0
5     C C C B C C C B Z 0
6     A B C A B C A Z 0 0
7     D B D B B B D Z 0 0
8     D B A D B D A Z 0 0
9     C A D B D A B D Z 0
10    D G A D G D A D Z 0
11    E E E F E E F Z 0 0
12    F A G F A G Z 0 0 0
13    E B A G B A G Z 0 0
14    B B B B B B B B Z 0
15    A G B E A A G B E Z
16    E B G E B G E B B Z
17    F F F F F F F F Z 0
18    A F E F A F E F Z 0
19    B F E G B F E G Z 0
Name: 1, dtype: object

In [3]:
full_lexicon = " ".join(target).split()
def create_lookup_tables(words):
    """
    Create lookup tables for vocabulary
    :param words: Input list of words
    :return: A tuple of dicts.  The first dict maps a vocab word to and integeter
             The second maps an integer back to to the vocab word
    """
    word_counts = Counter(words)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab, 1)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab
vocab_to_int, int_to_vocab = create_lookup_tables(full_lexicon)

In [4]:
d = []
for x in range(len(data)):
    d.append(data[x].split(" "))
for x in range(len(d)):
    for z in range(len(d[x])):
        if d[x][z] == 'A':
            d[x][z] = vocab_to_int['A']
        elif d[x][z] == 'B':
            d[x][z] = vocab_to_int['B']
        elif d[x][z] == 'C':
            d[x][z] = vocab_to_int['C']
        elif d[x][z] == 'D':
            d[x][z] = vocab_to_int['D']
        elif d[x][z] == 'E':
            d[x][z] = vocab_to_int['E']
        elif d[x][z] == 'F':
            d[x][z] = vocab_to_int['F']
        elif d[x][z] == 'G':
            d[x][z] = vocab_to_int['G']
        elif d[x][z] == 'Z':
            d[x][z] = vocab_to_int['Z']
d = [d]
d = np.array(d)
d = d.reshape(20, 1, 5) / 8

In [5]:
a = []
for x in range(len(target)):
    a.append(target[x].split(" "))
for x in range(len(a)):
    for z in range(len(a[x])):
        if a[x][z] == 'A':
            a[x][z] = int(vocab_to_int['A'])
        elif a[x][z] == 'B':
            a[x][z] = vocab_to_int['B']
        elif a[x][z] == 'C':
            a[x][z] = vocab_to_int['C']
        elif a[x][z] == 'D':
            a[x][z] = vocab_to_int['D']
        elif a[x][z] == 'E':
            a[x][z] = vocab_to_int['E']
        elif a[x][z] == 'F':
            a[x][z] = vocab_to_int['F']
        elif a[x][z] == 'G':
            a[x][z] = vocab_to_int['G']
        elif a[x][z] == 'Z':
            a[x][z] = vocab_to_int['Z']
        elif a[x][z] == '0':
            a[x][z] = 0

a = [a]
a = np.array(a)
a = a.reshape(20, 1, 10) / 8

In [6]:
d*8

array([[[2., 1., 2., 1., 4.]],

       [[2., 6., 2., 6., 4.]],

       [[2., 7., 2., 7., 4.]],

       [[6., 1., 6., 1., 4.]],

       [[2., 2., 2., 1., 4.]],

       [[6., 6., 6., 1., 4.]],

       [[2., 1., 6., 2., 4.]],

       [[7., 1., 7., 1., 4.]],

       [[7., 1., 2., 7., 4.]],

       [[6., 2., 7., 1., 4.]],

       [[7., 9., 2., 7., 4.]],

       [[8., 8., 8., 5., 4.]],

       [[5., 5., 2., 9., 4.]],

       [[8., 1., 2., 9., 4.]],

       [[1., 1., 1., 1., 4.]],

       [[2., 9., 1., 8., 4.]],

       [[8., 9., 1., 1., 4.]],

       [[5., 5., 5., 5., 4.]],

       [[2., 5., 8., 5., 4.]],

       [[1., 5., 8., 9., 4.]]])

In [7]:
a*8

array([[[2., 1., 2., 1., 1., 1., 2., 4., 0., 0.]],

       [[2., 6., 2., 6., 6., 6., 2., 4., 0., 0.]],

       [[2., 7., 2., 7., 7., 7., 2., 4., 0., 0.]],

       [[6., 1., 6., 1., 6., 1., 6., 1., 4., 0.]],

       [[2., 2., 2., 1., 2., 2., 2., 1., 4., 0.]],

       [[6., 6., 6., 1., 6., 6., 6., 1., 4., 0.]],

       [[2., 1., 6., 2., 1., 6., 2., 4., 0., 0.]],

       [[7., 1., 7., 1., 1., 1., 7., 4., 0., 0.]],

       [[7., 1., 2., 7., 1., 7., 2., 4., 0., 0.]],

       [[6., 2., 7., 1., 7., 2., 1., 7., 4., 0.]],

       [[7., 9., 2., 7., 9., 7., 2., 7., 4., 0.]],

       [[8., 8., 8., 5., 8., 8., 5., 4., 0., 0.]],

       [[5., 2., 9., 5., 2., 9., 4., 0., 0., 0.]],

       [[8., 1., 2., 9., 1., 2., 9., 4., 0., 0.]],

       [[1., 1., 1., 1., 1., 1., 1., 1., 4., 0.]],

       [[2., 9., 1., 8., 2., 2., 9., 1., 8., 4.]],

       [[8., 1., 9., 8., 1., 9., 8., 1., 1., 4.]],

       [[5., 5., 5., 5., 5., 5., 5., 5., 4., 0.]],

       [[2., 5., 8., 5., 2., 5., 8., 5., 4., 0.]],

       [[1.,

In [8]:
#splitting data into train and test sets. 3/4 train, 1/4 test.
x_train,x_test,y_train,y_test = train_test_split(d, a, test_size=0.2, shuffle=False, random_state=42)

In [9]:
#getting the right tensor shape for this was a BITCH
model = Sequential()  
model.add(LSTM(128, input_shape=(1, 5),return_sequences=True, activation = 'relu'))
model.add(LSTM(128, return_sequences=True, activation = 'relu'))
model.add(LSTM(128, return_sequences=True, activation = 'relu'))
model.add(LSTM(128, return_sequences=True, activation = 'relu'))
model.add(LSTM(128, return_sequences=True, activation = 'tanh'))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=True, activation = 'relu'))
model.add(LSTM(128, return_sequences=True, activation = 'relu'))
model.add(LSTM(128, return_sequences=True, activation = 'relu'))
model.add(LSTM(128, return_sequences=True, activation = 'relu'))
model.add(LSTM(128, return_sequences=True, activation = 'relu'))
model.add(LSTM(128, return_sequences=True, activation = 'sigmoid'))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=True, activation = 'relu'))
model.add(LSTM(128, return_sequences=True, activation = 'relu'))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True, activation = 'relu'))
model.add(LSTM(128, return_sequences=True, activation = 'relu'))
model.add(LSTM(128, return_sequences=True, activation = 'tanh'))
model.add(LSTM(128, return_sequences=True, activation = 'relu'))
model.add(LSTM(128, return_sequences=True, activation = 'relu'))
model.add(Dense(units=128))
model.add(Dense(units=128))
model.add(Dense(units=128))
model.add(Dense(units=10))
model.add(Dense(units=10))
model.add(Dense(units=10))
model.add(Activation('softmax'))



model.compile(loss='mean_absolute_error', optimizer='adam')
print(model.summary())

W0913 15:46:16.185493  7644 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0913 15:46:16.196464  7644 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0913 15:46:16.198480  7644 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0913 15:46:16.801033  7644 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0913 15:46:16.805023  7644 deprecation.py:506] From D:\Anaconda\lib\site-packages\keras\backend\t

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 128)            68608     
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 128)            131584    
_________________________________________________________________
lstm_3 (LSTM)                (None, 1, 128)            131584    
_________________________________________________________________
lstm_4 (LSTM)                (None, 1, 128)            131584    
_________________________________________________________________
lstm_5 (LSTM)                (None, 1, 128)            131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 128)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 1, 128)           

In [10]:
model.fit(d, a, epochs=25, batch_size=5, verbose=2,validation_data=(x_test, y_test))

W0913 15:46:19.051013  7644 deprecation.py:323] From D:\Anaconda\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0913 15:46:25.749111  7644 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 20 samples, validate on 4 samples
Epoch 1/25
 - 9s - loss: 0.3793 - val_loss: 0.4994
Epoch 2/25
 - 0s - loss: 0.3731 - val_loss: 0.4947
Epoch 3/25
 - 0s - loss: 0.3666 - val_loss: 0.4902
Epoch 4/25
 - 0s - loss: 0.3610 - val_loss: 0.4883
Epoch 5/25
 - 0s - loss: 0.3594 - val_loss: 0.4879
Epoch 6/25
 - 0s - loss: 0.3587 - val_loss: 0.4887
Epoch 7/25
 - 0s - loss: 0.3580 - val_loss: 0.4887
Epoch 8/25
 - 0s - loss: 0.3569 - val_loss: 0.4886
Epoch 9/25
 - 0s - loss: 0.3563 - val_loss: 0.4879
Epoch 10/25
 - 0s - loss: 0.3561 - val_loss: 0.4877
Epoch 11/25
 - 0s - loss: 0.3560 - val_loss: 0.4876
Epoch 12/25
 - 0s - loss: 0.3557 - val_loss: 0.4879
Epoch 13/25
 - 0s - loss: 0.3555 - val_loss: 0.4883
Epoch 14/25
 - 0s - loss: 0.3555 - val_loss: 0.4877
Epoch 15/25
 - 0s - loss: 0.3554 - val_loss: 0.4876
Epoch 16/25
 - 0s - loss: 0.3554 - val_loss: 0.4879
Epoch 17/25
 - 0s - loss: 0.3556 - val_loss: 0.4896
Epoch 18/25
 - 0s - loss: 0.3560 - val_loss: 0.4882
Epoch 19/25
 - 0s - loss: 0.35

In [11]:
guess = np.array([2, 1, 1, 1, 4])
guess = guess.reshape(1,1,5)

In [12]:
predict = model.predict_on_batch(guess)*8

In [20]:
array = np.around(predict[0][0])
print(array)
new = []

[1. 1. 1. 1. 1. 1. 1. 1. 0. 0.]


In [21]:
for z in range(len(array)):
    if array[z] == 1:
        new.append(int_to_vocab[1])
    elif array[z] == 2:
        new.append(int_to_vocab[2])
    elif array[z] == 3:
        new.append(int_to_vocab[3])
    elif array[z] == 4:
        new.append(int_to_vocab[4])
    elif array[z] == 5:
        new.append(int_to_vocab[5])
    elif array[z] == 6:
        new.append(int_to_vocab[6])
    elif array[z] == 7:
        new.append(int_to_vocab[7])
    elif array[z] == 8:
        new.append(int_to_vocab[8])
    elif array[z] == 0:
        new.append(0)

In [22]:
new

['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 0, 0]